<a href="https://colab.research.google.com/github/rahiakela/grokking-deep-learning/blob/13-introducing-automatic-optimization/13_introducing_automatic_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing automatic optimization: let’s build a deep learning framework

It’s extremely important for you to know what’s going on under the hood of these frameworks by implementing algorithms yourself (from scratch in NumPy). But now we’re going to transition into using a framework, because the networks you’ll be training next—long shortterm memory networks (LSTMs)—are very complex, and NumPy code describing their implementation is difficult to read, use, or debug (gradients are flying everywhere).

It’s exactly this code complexity that deep learning frameworks were created to mitigate. **Especially if you wish to train a neural network on a GPU (giving 10–100× faster training), a deep learning framework can significantly reduce code complexity (reducing errors and increasing development speed) while also increasing runtime performance.**

For these reasons, their use is nearly universal within the research community, and a thorough understanding of a deep learning framework will be essential on your journey toward becoming a user or researcher of deep learning.

This way, you’ll have no doubt about what frameworks do when
using them for complex architectures. Furthermore, building a small framework yourself should provide a smooth transition to using actual deep learning frameworks, because you’ll already be familiar with the API and the functionality underneath it.

**Abstractly, it eliminates the need to write code that you’d repeat multiple times. Concretely, the most beneficial pieces of a deep learning
framework are its support for automatic backpropagation and automatic optimization. These features let you specify only the forward propagation code of a model, with the framework taking care of backpropagation and weight updates automatically. Most frameworks even make the forward propagation code easier by providing high-level interfaces to common layers and loss functions.**

## Introduction to tensors

**Tensors are an abstract form of vectors and matrices.**

Up to this point, we’ve been working exclusively with vectors and matrices as the basic data structures for deep learning. Recall that **a matrix is a list of vectors, and a vector is a list of scalars (single numbers). A tensor is the abstract version of this form of nested lists of numbers. A vector is a one-dimensional tensor. A matrix is a two-dimensional tensor, and higher dimensions are referred to as n-dimensional tensors**. Thus, the beginning of a new deep learning framework is the construction of this basic type, which we’ll call Tensor:

In [1]:
import numpy as np

In [2]:
class Tensor(object):

  def __init__(self, data):
    self.data = np.array(data)

  def __add__(self, other):
    return Tensor(self.data + other.data)

  def __repr__(self):
    return str(self.data.__repr__())

  def __str__(self):
    return str(self.data.__str__())

In [3]:
x = Tensor([1, 2, 3, 4, 5])
print(x)

[1 2 3 4 5]


In [4]:
y = x + x
print(y)

[ 2  4  6  8 10]


Note that it stores all the numerical information in a NumPy array (self.data), and it supports one tensor operation (addition). Adding more operations is relatively simple: create more functions on the tensor class with the appropriate functionality.

## Introduction to automatic gradient computation(autograd)

**Stop! you performed backpropagation by hand.
Let’s make it automatic!**

You learned about derivatives. Since then, you’ve been computing derivatives
by hand for each neural network you train. Recall that this is done by moving backward through the neural network: **first compute the gradient at the output of the network, then use that result to compute the derivative at the next-to-last component, and so on until all weights in the architecture have correct gradients. This logic for computing gradients can also be added to the tensor object.**

In [5]:
class Tensor(object):

  def __init__(self, data, creators=None, creation_op=None):
    self.data = np.array(data)
    self.creation_op = creation_op
    self.creators = creators
    self.grad = None

  def backward(self, grad):
    self.grad = grad

    if (self.creation_op == 'add'):
      self.creators[0].backward(grad)
      self.creators[1].backward(grad)

  def __add__(self, other):
    return Tensor(self.data + other.data, creators=[self, other], creation_op='add')

  def __repr__(self):
    return str(self.data.__repr__())

  def __str__(self):
    return str(self.data.__str__())

In [6]:
x = Tensor([1, 2, 3, 4, 5])
y = Tensor([2, 2, 2, 2, 2])
print(x, y)

[1 2 3 4 5] [2 2 2 2 2]


In [7]:
z = x + y
print(z)

[3 4 5 6 7]


In [8]:
print(z.backward(Tensor(np.array([1, 1, 1, 1, 1]))))

None


This method introduces two new concepts. First, each tensor gets two new attributes. creators is a list containing any tensors used in the creation of the current tensor (which defaults to None). Thus, when the two tensors $x$ and $y$ are added together, $z$ has two creators, $x$ and $y$. creation_op is a related feature that stores the instructions creators used in the creation process. 

**Thus, performing $z = x + y$ creates a computation graph with
three nodes ($x$, $y$, and $z$) and two edges ($z -> x$ and $z -> y$). Each edge is labeled by the creation_op add. This graph allows you to recursively backpropagate gradients.**

<img src='https://github.com/rahiakela/img-repo/blob/master/grokking-deep-learning/node-graph.png?raw=1' width='800'/>

**The first new concept in this implementation is the automatic creation of this graph whenever you perform math operations. If you took $z$ and performed further operations, the graph would continue with whatever resulting new variables pointed back to $z$.**

**The second new concept introduced in this version of Tensor is the ability to use this graph to compute gradients.** When you call $z.backward()$, it sends the correct gradient for $x$ and $y$ given the function that was applied to create $z(add)$.

Looking at the graph, you place a vector of gradients (np.array([1,1,1,1,1])) on $z$, and then they’re applied to their parents. As you know, backpropagating through addition means also applying addition when backpropagating. 

In this case, because there’s only one gradient to add into $x$
or $y$, you copy the gradient from $z$ onto $x$ and $y$:




In [9]:
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


Perhaps the most elegant part of this form of autograd is that it works recursively as well, because each vector calls .backward() on all of its self.creators:

In [10]:
a = Tensor([1, 2, 3, 4, 5])
b = Tensor([2, 2, 2, 2, 2])
c = Tensor([5, 4, 3, 2, 1])
d = Tensor([-1, -2, -3, -4, -5])

e = a + b
f = c + d
g = e + f

In [11]:
g.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(a.grad)

[1 1 1 1 1]


## A quick checkpoint

**Everything in Tensor is another form of lessons already learned.**

Before moving on, I want to first acknowledge that even if it feels like a bit of a stretch or a heavy lift to think about gradients flowing over a graphical structure, this is nothing new compared to what you’ve already been working with. In the previous chapter on RNNs, you forward propagated in one direction and then back propagated across a (virtual graph) of activations.

In particular, this notion of a graph that gets built during forward propagation is called a dynamic computation graph because it’s built
on the fly during forward prop. This is the type of autograd present in newer deep learning frameworks such as DyNet and PyTorch. Older frameworks such as Theano and TensorFlow have what’s called a static computation graph, which is specified before forward propagation even begins.

In general, dynamic computation graphs are easier to write/experiment with, and static computation graphs have faster runtimes because of some fancy logic under the hood. But note that dynamic and static frameworks have lately been moving toward the middle, allowing dynamic graphs to compile to static ones (for faster runtimes) or allowing static graphs to be built dynamically (for easier experimentation). In the long run, you’re likely to end up with both. The primary difference is whether forward propagation is happening during graph construction or after the graph is already defined.

The main point, here, is to help prepare you for deep learning in the real world, where 10% (or less) of your time will be spent thinking up a new idea and 90% of your time will be spent figuring out how to get a deep learning framework to play nicely. Debugging these frameworks can be extremely difficult at times, because most bugs don’t raise an error and print out a stack trace. Most bugs lie hidden within the code, keeping the network from training as it should (even if it appears to be training somewhat).

## Tensors that are used multiple times

**The basic autograd has a rather pesky bug. Let’s squish it!**

Sometimes, during forward propagation, you’ll use the same tensor multiple times (the weights of a neural network), and thus multiple parts of the graph will backpropagate gradients into the same tensor. But the code will currently compute the incorrect gradient when backpropagating into a variable that was used multiple times (is the parent of multiple children).

In [12]:
a = Tensor([1, 2, 3, 4, 5])
b = Tensor([2, 2, 2, 2, 2])
c = Tensor([5, 4, 3, 2, 1])

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([2, 2, 2, 2, 2]))

[False False False False False]


In this example, the b variable is used twice in the process of creating f. Thus, its gradient should be the sum of two derivatives: [2,2,2,2,2]. Shown here is the resulting graph created by this chain of operations. Notice there are now two pointers pointing into b: so, it should be the sum of the gradient coming from both e and d.

<img src='https://github.com/rahiakela/img-repo/blob/master/grokking-deep-learning/node-graph-2.png?raw=1' width='800'/>

But the current implementation of Tensor merely overwrites each derivative with the previous. First, d applies its gradient, and then it gets overwritten with the gradient from e. We need to change the way gradients are written.

## Upgrading autograd to support multiuse tensors

**Add one new function, and update three old ones.**

This update to the Tensor object adds two new features. First, gradients can be accumulated so that when a variable is used more than once, it receives gradients from all children:

In [13]:
class Tensor(object):

  def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
    self.data = np.array(data)
    self.creation_op = creation_op
    self.creators = creators
    self.grad = None
    self.autograd = autograd
    self.children = {}

    if (id is None):
      id = np.random.randint(0, 100000)
    self.id = id

    if (creators is not None):
      for c in creators:
        # Keeps track of how many children a tensor has
        if (self.id not in c.children):
          c.children[self.id] = 1
        else:
          c.children[self.id] += 1

  def all_children_grads_accounted_for(self):
    # Checks whether a tensor has received the correct number of gradients from each child
    for id, cnt in self.children.items():
      if(cnt != 0):
        return False
      return True

  '''
  Notice that addition isn’t handled anywhere else in the class. The generic backpropagation
  logic is abstracted away so everything necessary for addition is defined in these two places.
  Note further that backpropagation logic calls .backward() two times, once for each variable
  that participated in the addition. Thus, the default setting in the backpropagation logic is to
  always backpropagate into every variable in the graph.
  '''
  def backward(self, grad, grad_origin=None):
    if (self.autograd):
      if (grad_origin is not None):
        # Checks to make sure you can backpropagate or whether you’re waiting for a gradient, in which case decrement the counter
        if (self.children[grad_origin.id] == 0):
          raise Exception('cannot backprop more than once')
        else:
          self.children[grad_origin.id] -= 1

      # Accumulates gradients from several children
      if (self.grad is None):
        self.grad = grad
      else:
        self.grad += grad

      if (self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
        if (self.creation_op == 'add'):  # Begins actual backpropagation
          self.creators[0].backward(self.grad, self)
          self.creators[1].backward(self.grad, self)

  def __add__(self, other):
    if (self.autograd and other.autograd):
      return Tensor(self.data + other.data, autograd=True, creators=[self, other], creation_op='add')
    else:
      return Tensor(self.data + other.data)

  def __repr__(self):
    return str(self.data.__repr__())

  def __str__(self):
    return str(self.data.__str__())

In [14]:
a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([2, 2, 2, 2, 2]))

[ True  True  True  True  True]


Additionally, you create a self.children counter that counts the number of gradients received from each child during backpropagation. This way, you also prevent a variable from accidentally backpropagating from the same child twice (which throws an exception).

Additionally, you create a self.children counter that counts the number of gradients received from each child during backpropagation. This way, you also prevent a variable from accidentally backpropagating from the same child twice (which throws an exception).

As mentioned previously, none of these concepts are new from a deep learning theory perspective; these are the kinds of engineering challenges that deep learning frameworks seek to face. More important, they’re the kinds of challenges you’ll face when debugging neural networks in a standard framework. Before moving on, take a moment to play around and get familiar with this code.

Try deleting different parts and seeing how it breaks in various ways. 
Try calling .backprop() twice.

In [15]:
a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([2, 2, 2, 2, 2]))

f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([2, 2, 2, 2, 2]))

[ True  True  True  True  True]


Exception: ignored

## Adding support for negation

**Let’s modify the support for addition to support negation.**

Now that addition is working, you should be able to copy and paste the addition code, create a few modifications, and add autograd support for negation. 

Nearly everything is identical. You don’t accept any parameters so the parameter “other” has been removed in several places.

Because the __neg__ function has only one creator, you end up calling .backward() only once.

Let’s try it. Modifications from the __add__ function are added:

In [16]:
class Tensor(object):

  def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
    self.data = np.array(data)
    self.creation_op = creation_op
    self.creators = creators
    self.grad = None
    self.autograd = autograd
    self.children = {}

    if (id is None):
      id = np.random.randint(0, 100000)
    self.id = id

    if (creators is not None):
      for c in creators:
        # Keeps track of how many children a tensor has
        if (self.id not in c.children):
          c.children[self.id] = 1
        else:
          c.children[self.id] += 1

  def all_children_grads_accounted_for(self):
    # Checks whether a tensor has received the correct number of gradients from each child
    for id, cnt in self.children.items():
      if(cnt != 0):
        return False
      return True

  '''
  Notice that addition isn’t handled anywhere else in the class. The generic backpropagation
  logic is abstracted away so everything necessary for addition is defined in these two places.
  Note further that backpropagation logic calls .backward() two times, once for each variable
  that participated in the addition. Thus, the default setting in the backpropagation logic is to
  always backpropagate into every variable in the graph.
  '''
  def backward(self, grad, grad_origin=None):
    if (self.autograd):
      if (grad_origin is not None):
        # Checks to make sure you can backpropagate or whether you’re waiting for a gradient, in which case decrement the counter
        if (self.children[grad_origin.id] == 0):
          raise Exception('cannot backprop more than once')
        else:
          self.children[grad_origin.id] -= 1

      # Accumulates gradients from several children
      if (self.grad is None):
        self.grad = grad
      else:
        self.grad += grad

      if (self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
        if (self.creation_op == 'add'):  # Begins actual backpropagation for addition 
          self.creators[0].backward(self.grad, self)
          self.creators[1].backward(self.grad, self)
        if (self.creation_op == 'neg'):  # Begins actual backpropagation for negation
          self.creators[0].backward(self.grad.__neg__())

  def __add__(self, other):
    if (self.autograd and other.autograd):
      return Tensor(self.data + other.data, autograd=True, creators=[self, other], creation_op='add')
    else:
      return Tensor(self.data + other.data)

  def __neg__(self):
    if (self.autograd):
      return Tensor(self.data * -1, autograd=True, creators=[self], creation_op='neg')
    else:
      return Tensor(self.data * -1)

  def __repr__(self):
    return str(self.data.__repr__())

  def __str__(self):
    return str(self.data.__str__())

In [17]:
a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)

d = a + (-b)
e = (-b) + c
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad.data == np.array([-2, -2, -2, -2, -2]))

[ True  True  True  True  True]


When you forward propagate using -b instead of b, the gradients that are backpropagated have a flipped sign as well. Furthermore, you don’t have to change anything about the general backpropagation system to make this work. You can create new functions as you need them.

## Adding support for additional functions

**Subtraction, multiplication, sum, expand, transpose, and matrix multiplication**

Using the same ideas you learned for addition and negation, let’s add the forward and backpropagation logic for several additional functions and You can now add the corresponding backpropagation logic to the .backward() method::

In [20]:
class Tensor(object):

  def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
    self.data = np.array(data)
    self.creation_op = creation_op
    self.creators = creators
    self.grad = None
    self.autograd = autograd
    self.children = {}

    if (id is None):
      id = np.random.randint(0, 100000)
    self.id = id

    if (creators is not None):
      for c in creators:
        # Keeps track of how many children a tensor has
        if (self.id not in c.children):
          c.children[self.id] = 1
        else:
          c.children[self.id] += 1

  def all_children_grads_accounted_for(self):
    # Checks whether a tensor has received the correct number of gradients from each child
    for id, cnt in self.children.items():
      if(cnt != 0):
        return False
      return True

  '''
  Notice that addition isn’t handled anywhere else in the class. The generic backpropagation
  logic is abstracted away so everything necessary for addition is defined in these two places.
  Note further that backpropagation logic calls .backward() two times, once for each variable
  that participated in the addition. Thus, the default setting in the backpropagation logic is to
  always backpropagate into every variable in the graph.
  '''
  def backward(self, grad, grad_origin=None):
    if (self.autograd):
      if (grad_origin is not None):
        # Checks to make sure you can backpropagate or whether you’re waiting for a gradient, in which case decrement the counter
        if (self.children[grad_origin.id] == 0):
          raise Exception('cannot backprop more than once')
        else:
          self.children[grad_origin.id] -= 1

      # Accumulates gradients from several children
      if (self.grad is None):
        self.grad = grad
      else:
        self.grad += grad

      if (self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
        if (self.creation_op == 'add'):  # Begins actual backpropagation for addition 
          self.creators[0].backward(self.grad, self)
          self.creators[1].backward(self.grad, self)
        if (self.creation_op == 'neg'):  # Begins actual backpropagation for negation
          self.creators[0].backward(self.grad.__neg__())
        if (self.creation_op == 'sub'):  # Begins actual backpropagation for subtraction
          new = Tensor(self.grad.data)
          self.creators[0].backward(new, self)
          new = Tensor(self.grad.__neg__().data)
          self.creators[1].backward(new, self)
        if (self.creation_op == 'mul'):  # Begins actual backpropagation for multiplication
          new = self.grad * self.creators[1]
          self.creators[0].backward(new, self)
          new = self.grad * self.creators[0]
          self.creators[1].backward(new, self)
        if (self.creation_op == 'mm'):  # Begins actual backpropagation for matrix multiplication
          activation = self.creators[0] # Usually an activation
          weights = self.creators[1]    # Usually an weight matrix
          new = self.grad.mm(weights.transpose())
          activation.backward(new)
          new = self.grad.transpose().mm(activation).transpose()
          weights.backward(new)
        if (self.creation_op == 'transpose'):  # Begins actual backpropagation for transpose
          self.creators[0].backward(self.grad.transpose())
        if ('sum' in self.creation_op):  # Begins actual backpropagation for sum
          dim = int(self.creation_op.split('_')[1])
          ds = self.creators[0].data.shape[dim]
          self.creators[0].backward(self.grad.expand(dim, ds))
        if ('expand' in self.creation_op):  # Begins actual backpropagation for expand
          dim = int(self.creation_op.split('_')[1])
          self.creators[0].backward(self.grad.sum(dim))

  def __add__(self, other):
    if (self.autograd and other.autograd):
      return Tensor(self.data + other.data, autograd=True, creators=[self, other], creation_op='add')
    else:
      return Tensor(self.data + other.data)

  def __neg__(self):
    if (self.autograd):
      return Tensor(self.data * -1, autograd=True, creators=[self], creation_op='neg')
    else:
      return Tensor(self.data * -1)
  
  def __sub__(self, other):
    if (self.autograd and other.autograd):
      return Tensor(self.data - other.data, autograd=True, creators=[self, other], creation_op='sub')
    return Tensor(self.data - other.data)

  def __mul__(self, other):
    if (self.autograd and other.autograd):
      return Tensor(self.data * other.data, autograd=True, creators=[self, other], creation_op='mul')
    return Tensor(self.data * other.data)

  def sum(self, dim):
    if (self.autograd):
      return Tensor(self.data.sum(dim), autograd=True, creators=[self], creation_op='sum_' + str(dim))
    return Tensor(self.data.sum(dim))

  def expand(self, dim, copies):
    trans_cmd = list(range(0, len(self.data.shape)))
    trans_cmd.insert(dim, len(self.data.shape))

    new_shape = list(self.data.shape) + [copies]
    new_data = self.data.repeat(copies).reshape(new_shape)
    new_data = new_data.transpose(trans_cmd)

    if (self.autograd):
      return Tensor(new_data, autograd=True, creators=[self], creation_op='expand_' + str(dim))
    return Tensor(new_data)

  def transpose(self):
    if (self.autograd):
      return Tensor(self.data.transpose(), autograd=True, creators=[self], creation_op='transpose')
    return Tensor(self.data.transpose())

  def mm(self, x):
    if (self.autograd):
      return Tensor(self.data.dot(x), autograd=True, creators=[self, x], creation_op='mm')
    return Tensor(self.data.dot(x.data))

  def __repr__(self):
    return str(self.data.__repr__())

  def __str__(self):
    return str(self.data.__str__())

We’ve previously discussed the derivatives for all these functions, although sum and expand might seem foreign because they have new names. sum performs addition across a dimension of the tensor; in other words, say you have a 2 × 3 matrix called x:

In [21]:
x = Tensor(np.array([
  [1, 2, 3],
  [4, 5, 6]                
]))

The .sum(dim) function sums across a dimension.

Where x.sum(0) will result in a 1 × 3 matrix (a length 3 vector), summing columns values.

whereas x.sum(1) will result in a 2 × 1 matrix (a length 2 vector), summing rows values:

In [22]:
x.sum(0) 

array([5, 7, 9])

In [23]:
x.sum(1)

array([ 6, 15])

You use expand to backpropagate through a .sum(). It’s a function that copies data along a dimension. 

Given the same matrix x, copying along the first dimension gives two copies of the tensor:

In [24]:
x.expand(dim=0, copies=4)

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

To be clear, whereas .sum() removes a dimension (2 × 3 -> just 2 or 3), expand adds a dimension. The 2 × 3 matrix becomes 4 × 2 × 3. You can think of this as a list of four tensors, each of which is 2 × 3. 

But if you expand to the last dimension, it copies along the last dimension, so each entry in the original tensor becomes a list of entries instead:

In [25]:
x.expand(dim=2, copies=4)

array([[[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]],

       [[4, 4, 4, 4],
        [5, 5, 5, 5],
        [6, 6, 6, 6]]])

Thus, when you perform .sum(dim=1) on a tensor with four entries in that dimension, you need to perform .expand(dim=1, copies=4) to the gradient when you backpropagate it.

In [26]:
x.expand(dim=1, copies=4)

array([[[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]],

       [[4, 5, 6],
        [4, 5, 6],
        [4, 5, 6],
        [4, 5, 6]]])

The gradients start at the end of the network. You then move the error signal backward through the network by calling functions that correspond to the functions used to move activations forward through the network. If the last operation was a matrix multiplication (and it was), you backpropagate by performing matrix multiplication (dot) on the transposed matrix.

## Using autograd to train a neural network